In [ ]:
!gcloud auth login
# !gcloud projects create synapse-copyright-detection --name="Synapse Copyright Detection"

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=Tz3vPkHeAeWYRsy5mEmtjsSLk6efCS&prompt=consent&token_usage=remote&access_type=offline&code_challenge=o-5afLkQuaq2-W8HuPuo7QN0kfwT-q0QFEmE6hHcXgU&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AanRRrswcfHciqMaf2LkFjBOOUYTyPViOiOS4cWHZohl1eJcV7il753k0ZIUOBR2mpbz1Q

You are now logged in as [aryan20102002@gmail.com].
Your current projec

In [ ]:
!gcloud config set project synapse-copyright-detection

Updated property [core/project].


In [ ]:
# Install required libraries
!pip install transformers torch torchaudio librosa opencv-python-headless faiss-cpu
!pip install google-cloud-storage

# Authenticate and enable Google Cloud SDK for YouTube-8M dataset
!gcloud auth application-default login

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 72.5 MB/s eta 0:00:00
Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=nqB5Eph0nNu41cHqdFF4fO2XfE8qgB&prompt=consent&token_usage=remote&access_type=offline&code_challenge=KnSHAo59EHcfuY5cNM3cIXf7Uux_S_DgWnJ--T6xudw&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AanRRruTPwh2FPb_yZ5UktymDJcqQgDgDj9kAtbFq5mQT_IbNwvGPGMQICDkt4HRUd_gKQ

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will

In [2]:
# !mkdir -p ~/data/yt8m/frame; cd ~/data/yt8m/frame
! curl data.yt8m.org/download.py | shard=1,300 partition=2/frame/train mirror=us python
# !curl data.yt8m.org/download.py | shard=1,100 partition=2/frame/validate mirror=us python
# !curl data.yt8m.org/download.py | shard=1,100 partition=2/frame/test mirror=us python

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4450  100  4450    0     0  17234      0 --:--:-- --:--:-- --:--:-- 17248
Resuming Download ...
Files remaining 9
Downloading: train3622.tfrecord
>> Downloading http://us.data.yt8m.org/2/frame/train/train6A.tfrecord 100.0%Succesfully downloaded train3622.tfrecord 276401566 bytes. 69.5%
Successfully downloaded train3622.tfrecord


Downloading: train3519.tfrecord
>> Downloading http://us.data.yt8m.org/2/frame/train/train4V.tfrecord 100.0%Succesfully downloaded train3519.tfrecord 265500722 bytes.
Successfully downloaded train3519.tfrecord


Downloading: train2791.tfrecord
>> Downloading http://us.data.yt8m.org/2/frame/train/trainTb.tfrecord 100.0%Succesfully downloaded train2791.tfrecord 279130176 bytes.
Successfully downloaded train2791.tfrecord


Downloading: train0477.tfrecord
>> Downloading http://us.data.yt8m.org/2/frame/tr

In [3]:
! python -m venv venv
! source venv/bin/activate
! pip install torch transformers opencv-python tensorflow librosa h5py tqdm numpy Pillow


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [4]:
! pip install setuptools


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [5]:
import torch
from transformers import CLIPProcessor, CLIPModel
import cv2
import numpy as np
import tensorflow as tf
from typing import Dict, List, Generator, Union
from tqdm import tqdm
import os
import h5py
import multiprocessing
from concurrent.futures import ThreadPoolExecutor
from PIL import Image
import logging
import librosa

ModuleNotFoundError: No module named 'torch'

In [4]:
class YT8MDataset:
    def __init__(self, data_dir: str):
        self.data_dir = data_dir
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s'
        )
        self.logger = logging.getLogger(__name__)
        self.tfrecord_files = self._get_tfrecord_files()

    def _get_tfrecord_files(self) -> List[str]:
        files = [os.path.join(self.data_dir, f) for f in os.listdir(self.data_dir)
                if f.endswith('.tfrecord')]
        self.logger.info(f"Found {len(files)} TFRecord files")
        return files

    def parse_tfrecord(self, example_proto):
        context_features = {
            'id': tf.io.FixedLenFeature([], tf.string),
            'labels': tf.io.VarLenFeature(tf.int64)
        }

        sequence_features = {
            'rgb': tf.io.FixedLenSequenceFeature([], tf.string),
            'audio': tf.io.FixedLenSequenceFeature([], tf.string)
        }

        try:
            context, sequence = tf.io.parse_single_sequence_example(
                example_proto,
                context_features=context_features,
                sequence_features=sequence_features
            )

            return context, sequence

        except tf.errors.InvalidArgumentError as e:
            self.logger.error(f"Error parsing TFRecord: {str(e)}")
            return None

    def decode_features(self, parsed_data):
        if parsed_data is None:
            return None

        try:
            context, sequence = parsed_data

            # Decode video ID
            video_id = context['id'].numpy().decode('utf-8')

            # Decode labels
            labels = tf.sparse.to_dense(context['labels']).numpy()

            # Decode RGB and audio features
            rgb_features = []
            audio_features = []

            for rgb_bytes in sequence['rgb']:
                rgb_frame = np.frombuffer(rgb_bytes.numpy(), dtype=np.uint8)
                rgb_frame = rgb_frame.reshape(-1, 1024)
                rgb_features.append(rgb_frame)

            for audio_bytes in sequence['audio']:
                audio_frame = np.frombuffer(audio_bytes.numpy(), dtype=np.uint8)
                audio_frame = audio_frame.reshape(-1, 128)
                audio_features.append(audio_frame)

            return {
                'id': video_id,
                'labels': labels,
                'rgb': np.array(rgb_features),
                'audio': np.array(audio_features)
            }

        except Exception as e:
            self.logger.error(f"Error decoding features: {str(e)}")
            return None

    def __iter__(self) -> Generator:
        for tfrecord_file in self.tfrecord_files:
            self.logger.info(f"Processing file: {tfrecord_file}")
            try:
                dataset = tf.data.TFRecordDataset(tfrecord_file)
                for raw_record in dataset:
                    features = self.parse_tfrecord(raw_record)
                    decoded_features = self.decode_features(features)
                    if decoded_features is not None:
                        yield decoded_features
            except Exception as e:
                self.logger.error(f"Error processing file {tfrecord_file}: {str(e)}")
                continue

In [5]:
class MP4Processor:
    def __init__(self, num_frames: int = 10):
        self.num_frames = num_frames
        self.logger = logging.getLogger(__name__)
        self.inception_model = self.initialize_inception_model()

    def initialize_inception_model(self):
        inception_base = tf.keras.applications.InceptionV3(
            include_top=False,
            weights='imagenet',
            input_shape=(299, 299, 3),
            pooling='avg'
        )
        inception_base.trainable = False

        model = tf.keras.Sequential([
            inception_base,
            tf.keras.layers.Dense(1024),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Activation('relu')
        ])

        model.compile(optimizer='adam', loss='mse')
        return model

    def extract_features(self, video_path: str) -> Dict[str, np.ndarray]:
        """Extract RGB and audio features from MP4 file"""
        frames = self._extract_frames(video_path)
        rgb_features = self._process_frames(frames)
        audio_features = self._extract_audio(video_path)

        return {
            'rgb': rgb_features,
            'audio': audio_features
        }

    def _extract_frames(self, video_path: str) -> List[np.ndarray]:
        """Extract evenly spaced frames from video"""
        frames = []
        cap = cv2.VideoCapture(video_path)

        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        frame_indices = np.linspace(0, total_frames-1, self.num_frames, dtype=int)

        for idx in frame_indices:
            cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
            ret, frame = cap.read()
            if ret:
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frames.append(frame_rgb)

        cap.release()
        return frames

    def _process_frames(self, frames: List[np.ndarray]) -> np.ndarray:
        """Process frames using InceptionV3 to match YouTube-8M format"""
        processed_frames = []
        for frame in frames:
            try:
                frame_resized = tf.image.resize(frame, (299, 299))
                frame_preprocessed = tf.keras.applications.inception_v3.preprocess_input(
                    frame_resized
                )
                frame_batch = tf.expand_dims(frame_preprocessed, 0)
                features = self.inception_model(frame_batch, training=False)
                features_normalized = tf.nn.l2_normalize(features, axis=1)
                processed_frames.append(features_normalized[0])
            except Exception as e:
                self.logger.error(f"Error processing frame: {str(e)}")
                continue

        if not processed_frames:
            raise ValueError("No frames were successfully processed")

        return np.stack(processed_frames)

    def _extract_audio(self, video_path: str) -> np.ndarray:
        """Extract audio features to match YouTube-8M format"""
        y, sr = librosa.load(video_path, sr=16000, mono=True)
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=128)
        mfccs_scaled = np.mean(mfccs.T, axis=0)
        return mfccs_scaled.reshape(1, -1)

In [6]:
class VideoCopyrightDetector:
    def __init__(self, use_gpu: bool = True):
        self.device = torch.device('cuda' if use_gpu and torch.cuda.is_available() else 'cpu')
        self.clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(self.device)
        self.clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
        self.mp4_processor = MP4Processor()

        self.visual_similarity_threshold = 0.85
        self.audio_similarity_threshold = 0.85

        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s'
        )
        self.logger = logging.getLogger(__name__)

        self.embedding_cache = {}

    def precompute_embeddings(self, dataset: YT8MDataset, cache_file: str = 'yt8m_embeddings_cache.h5'):
        if os.path.exists(cache_file):
            self.logger.info("Loading existing embedding cache...")
            with h5py.File(cache_file, 'r') as f:
                for video_id in f.keys():
                    self.embedding_cache[video_id] = {
                        'visual': torch.tensor(f[video_id]['visual'][()]),
                        'audio': torch.tensor(f[video_id]['audio'][()])
                    }
            return

        self.logger.info("Precomputing embeddings for dataset...")
        with h5py.File(cache_file, 'w') as cache:
            for features in tqdm(dataset):
                video_id = features['id']
                try:
                    visual_embedding = torch.tensor(features['rgb'])
                    audio_embedding = torch.tensor(features['audio'])

                    video_group = cache.create_group(video_id)
                    video_group.create_dataset('visual', data=visual_embedding.numpy())
                    video_group.create_dataset('audio', data=audio_embedding.numpy())

                    self.embedding_cache[video_id] = {
                        'visual': visual_embedding,
                        'audio': audio_embedding
                    }
                except Exception as e:
                    self.logger.error(f"Error processing video {video_id}: {str(e)}")

    def compare_mp4_with_dataset(self, mp4_path: str) -> List[Dict]:
        self.logger.info(f"Processing query video: {mp4_path}")
        query_features = self.mp4_processor.extract_features(mp4_path)

        query_features = {
            'id': os.path.basename(mp4_path),
            'rgb': query_features['rgb'],
            'audio': query_features['audio']
        }

        return self._compare_features(query_features)

    def _compare_features(self, query_features: Dict) -> List[Dict]:
        query_visual = torch.tensor(query_features['rgb']).to(self.device)
        query_audio = torch.tensor(query_features['audio']).to(self.device)

        results = []
        with ThreadPoolExecutor(max_workers=multiprocessing.cpu_count()) as executor:
            futures = []

            for video_id, embeddings in self.embedding_cache.items():
                future = executor.submit(
                    self._compute_similarity,
                    query_visual,
                    query_audio,
                    embeddings['visual'].to(self.device),
                    embeddings['audio'].to(self.device),
                    video_id
                )
                futures.append(future)

            for future in tqdm(futures, desc="Comparing with dataset"):
                result = future.result()
                if result['is_likely_copy']:
                    results.append(result)

        results.sort(key=lambda x: max(x['visual_similarity'], x['audio_similarity']),
                    reverse=True)
        return results

    def _compute_similarity(self, query_visual, query_audio, dataset_visual, dataset_audio, video_id):
    # Convert tensors to float for mean and similarity computations
        query_visual = query_visual.float()
        query_audio = query_audio.float()
        dataset_visual = dataset_visual.float()
        dataset_audio = dataset_audio.float()

        visual_similarity = float(torch.nn.functional.cosine_similarity(
            query_visual.mean(dim=0),
            dataset_visual.mean(dim=0),
            dim=0
        ))

        audio_similarity = float(torch.nn.functional.cosine_similarity(
            query_audio.mean(dim=0),
            dataset_audio.mean(dim=0),
            dim=0
        ))

        return {
            'video_id': video_id,
            'visual_similarity': visual_similarity,
            'audio_similarity': audio_similarity,
            'is_likely_copy': (visual_similarity > self.visual_similarity_threshold or
                             audio_similarity > self.audio_similarity_threshold),
            'details': self._generate_similarity_details(visual_similarity, audio_similarity)
        }


    def _generate_similarity_details(self, visual_sim, audio_sim):
        details = []
        if visual_sim > self.visual_similarity_threshold:
            details.append(f"High visual similarity detected: {visual_sim:.2f}")
        if audio_sim > self.audio_similarity_threshold:
            details.append(f"High audio similarity detected: {audio_sim:.2f}")
        return details

In [7]:
def main():
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s'
    )
    logger = logging.getLogger(__name__)

    try:
        detector = VideoCopyrightDetector(use_gpu=True)

        dataset_path = "./"
        dataset = YT8MDataset(dataset_path)

        detector.precompute_embeddings(dataset)

        query_video_path = "./test-video.mp4"

        results = detector.compare_mp4_with_dataset(query_video_path)

        print(f"\nFound {len(results)} potential matches:")
        for result in results:
            print(f"\nVideo ID: {result['video_id']}")
            print(f"Visual Similarity: {result['visual_similarity']:.2f}")
            print(f"Audio Similarity: {result['audio_similarity']:.2f}")
            print("Details:", "\n".join(result['details']))

    except Exception as e:
        logger.error(f"Error in main execution: {str(e)}")
        raise

In [8]:
if __name__ == "__main__":
    main()

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 32s 0us/step


2024-12-18 17:20:04,989 - ERROR - Error in main execution: [Errno 2] No such file or directory: '/content/'


FileNotFoundError: [Errno 2] No such file or directory: '/content/'